In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from visualize import PyramidVisualizer, get_dataset, simple_visualizer
from metroem.aligner import Aligner
from collections import defaultdict
import random

In [69]:
import modelhouse
import json

iter_count = 500

model_path = "gs://corgie/models/pyramid_m4m6m9/0_mip7in_mip9module"
aligners = defaultdict(lambda: {})


aligners[4]['noopt'] = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
         "finetune": True, "finetune_iter": 1, "checkpoint_name": "metric_net",
         "finetune_sm": 10
    })
)

aligners[7]['without_net'] = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
         "finetune": True, "finetune_iter": iter_count, "checkpoint_name": "metric_net",
         "finetune_sm": 3, "ignore_net_output": True
    })
)

aligners[7]['with_net'] = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
         "finetune": True, "finetune_iter": iter_count, "checkpoint_name": "metric_net",
         "finetune_sm": 3, "ignore_net_output": False
    })
)

model_path = "gs://corgie/models/pyramid_m4m6m9/1_mip5in_mip6module"

aligners[5]['without_net'] = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
         "finetune": True, "finetune_iter": iter_count, "checkpoint_name": "metric_net",
         "finetune_sm": 4, "ignore_net_output": True
    })
)

aligners[5]['with_net'] = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
         "finetune": True, "finetune_iter": iter_count, "checkpoint_name": "metric_net",
         "finetune_sm": 4, "ignore_net_output": False
    })
)

model_path = "gs://corgie/models/pyramid_m4m6m9/2_mip4in_mip4module"

aligners[4]['without_net'] = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
         "finetune": True, "finetune_iter": iter_count, "checkpoint_name": "metric_net",
         "finetune_sm": 6, "ignore_net_output": True
    })
)

aligners[4]['with_net'] = modelhouse.load_model(
    path=model_path,
    params=json.dumps({
         "finetune": True, "finetune_iter": iter_count, "checkpoint_name": "metric_net",
         "finetune_sm": 6, "ignore_net_output": False
    })
)



In [65]:
import cloudvolume as cv
import torch

img_path = {}
img_path['raw'] = 'gs://corgie_package/minie/test_x1/img/unaligned_normalized_defects'
img_path['aligned_m7'] = 'gs://corgie_package/tmp/minnie/m7_x3/img/img_aligned'
img_path['aligned_m5'] = 'gs://corgie_package/tmp/minnie/m5_x3/img/img_aligned'


z_s = 17501
z_e = 17509

x_s = 150000
x_e = 210000
y_s = 150000
y_e = 210000

def get_srctgt(cv_path, x_start, y_start, z, img_mip, x_size=1024, y_size=1024):
    img_cv = cv.CloudVolume(cv_path, mip=img_mip, parallel=True, progress=False, bounded=False, fill_missing=True)
    
    x_size *= 2**img_mip
    y_size *= 2**img_mip

    img1_np = img_cv[(x_start)//2**img_mip:(x_start+x_size)//2**img_mip, (y_start)//2**img_mip:(y_start+y_size)//2**img_mip, z:z+1].squeeze()
    img2_np = img_cv[(x_start)//2**img_mip:(x_start+x_size)//2**img_mip, (y_start)//2**img_mip:(y_start+y_size)//2**img_mip, z+1:z+2].squeeze()

    img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
    img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)
    
    img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
    img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)

    #src_tgt_t = torch.cat([img1_t, img2_t], 1)
    return img1_t, img2_t

In [66]:
x = random.randint(x_s, x_e)
y = random.randint(y_s, y_e)
z = random.randint(z_s, z_e)

src, tgt = get_srctgt(
    img_path['aligned_m7'],
    x, y, z, 
    img_mip=5
)
simple_visualizer().visualize([src, tgt])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1), value=0), IntText(value=1, descripti…

In [67]:
with_net_losses = aligners[5]['with_net'](src_img=src, tgt_img=tgt)
#pred_res[:, :, 20:-20, 20:-20].abs().max()


0.08483099937438965secs for net
0.9166306 0.61306614 0.30356446
New best: 448, No impr: 0, Iter: 499
0.3616344 0.22553119 0.13610321
7.179007053375244


In [68]:
without_net_losses = aligners[5]['without_net'](src_img=src, tgt_img=tgt)
#pred_res[:, :, 20:-20, 20:-20].abs().max()


0.07422757148742676secs for net
0.9166306 0.61306614 0.30356446
New best: 466, No impr: 0, Iter: 499
0.36170655 0.22561175 0.13609482
7.179155349731445


In [54]:
field = aligners[4]['noopt'](src_img=src, tgt_img=tgt)
field.max()

0.07712221145629883secs for net


field(2.8730, device='cuda:0')

In [55]:

simple_visualizer().visualize([src, field.from_pixels()(src), tgt])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2), value=0), IntText(value=1, descri…

In [24]:
simple_visualizer().visualize([pred_res, pred_res.from_pixels()(img1_t), img1_t, img2_t])

NameError: name 'pred_res' is not defined

In [6]:
import random
import torch
import numpy as np

import cloudvolume as cv
import torch

from visualize import PyramidVisualizer, get_dataset, simple_visualizer
from metroem.aligner import Aligner

#raw_img_path = 'gs://corgie_package/paper/basil_data_x0/aligned_x0/img/img_full_x0_rend'
raw_img_path = 'https://s3-hpcrc.rc.princeton.edu/fafbv15-em/v1'
raw_img_path = 'matrix://seamless-paper/prod_x3/normed/img/img_norm'


img_mip = 3

z = 500
x_start = 115000
y_start = 50000

x_size = 1024 * 2**img_mip
y_size = x_size
pad = 64 * 2**3
#
#img_cv = cv.CloudVolume(raw_img_path, mip=img_mip, parallel=True, progress=False)
#masked_img_cv = cv.CloudVolume(masked_img_path, mip=masked_img_mip, parallel=True, progress=False)
#field_cv = cv.CloudVolume(field_path, mip=field_mip, parallel=True, progress=False)
img_cv = cv.CloudVolume(raw_img_path, mip=img_mip, parallel=True, progress=False)
img1_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z:z+1].squeeze()
img2_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z+1:z+2].squeeze()

In [13]:
f"X: {(x_start-pad)//2**img_mip}-{(x_start+x_size+pad)//2**img_mip}" + f"  Y: {(y_start-pad)//2**img_mip}-{(y_start+y_size+pad)//2**img_mip}"

'X: 14311-15463  Y: 6186-7338'

In [15]:
img1_t = torch.tensor(img2_np, device='cuda').unsqueeze(0).unsqueeze(0)
img2_t = torch.tensor(img1_np, device='cuda').unsqueeze(0).unsqueeze(0)

src_tgt_t = torch.cat([img1_t, img2_t], 1)
pred_res = aligner(src_img=img1_t, tgt_img=img2_t)

In [16]:
img1_t.mean()

tensor(0.1529, device='cuda:0')

In [17]:
pred_res.max()

tensor(6.0104, device='cuda:0')

In [18]:
simple_visualizer().visualize([pred_res, pred_res.from_pixels()(img1_t), img1_t, img2_t])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3), value=0), IntText(value=1, des…

In [19]:

simple_visualizer().visualize([pred_res, pred_res.from_pixels()(img1_t), img1_t, img2_t])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3), value=0), IntText(value=1, des…

In [20]:
import metroem
from metroem.helpers import get_np
mask = (img1_np != 0).astype(np.float32)
closed = metroem.masks.closing(mask, n=40).astype(np.float32)
closed_2 = ndimage.binary_closing(mask, iterations=40).astype(np.int)
simple_visualizer().visualize([get_np(src_defects).astype(np.float32), get_np(src_zeros).astype(np.float32), (mask == 0) * closed, closed, closed_2, img1_np, mask, closed])

NameError: name 'ndimage' is not defined

In [ ]:
ndimage.binary_closing(a).astype(np.int)

In [19]:
a = np.zeros((5,5), dtype=np.int)

/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [20]:
a[1:-1, 1:-1] = 1; a[2,2] = 0

In [21]:
a

array([[0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0]])

In [22]:
simple_visualizer().visualize([a])

interactive(children=(ToggleButtons(description='Image:', options=(0,), value=0), IntText(value=1, description…

NameError: name 'ndimage' is not defined

In [24]:
from scipy import ndimage

In [26]:
once = ndimage.binary_closing(a).astype(np.int)

/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [27]:
simple_visualizer().visualize([once])

interactive(children=(ToggleButtons(description='Image:', options=(0,), value=0), IntText(value=1, description…